In [ ]:
import tensorflow as tf
import numpy as np
import random
import os
import glob
import cv2
import sys

### 生成图片信息文件image_inormation.txt
内容格式：图片文件完整路径[blank]label   
例：`F:/JupyterN/ELE_Contest/bin/utils/aug_images/4/11465213433_847c4fa261.jpg 4`


In [ ]:
aug_path = "F:/JupyterN/ELE_Contest/bin/utils/aug_images/"
with open('./image_information.txt', 'w') as f:
    for i in range(1, 6):
        for img_info in os.listdir(aug_path + '%d/'%i):
            f.write(aug_path + '%d/'%i + img_info + ' ' + str(i) + '\n')

# shuffle the images info
img_info = []
with open('./image_information.txt', 'r') as f:
    for line in f:
        img_info.append(line)

# 拆分成6个图片信息文件，用于生成6个tfrecords文件
random.shuffle(img_info)
for i in range(6):
    with open('./image_information_%d.txt'%i, 'w') as f:
        for j in img_info[i*1000 : i*1000 + 1000]:
            f.write(j)

### 转换成tfrecords文件

In [ ]:
def load_image(addr):
    # read an image and resize to (224, 224)
    # cv2 load images as BGR, convert it to RGB
    img = cv2.imread(addr)
    img = cv2.resize(img, (224, 224), interpolation=cv2.INTER_CUBIC)
    b, g, r = cv2.split(img)
    img = cv2.merge((g, g, g))
    img = img.astype(np.float32)
    return img

In [ ]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
for i in range(6):
    train_filename = 'train_%d.tfrecords'%i  # address to save the TFRecords file
    # open the TFRecords file
    writer = tf.python_io.TFRecordWriter(train_filename)
    imgdir = './image_information_%d.txt'%i
    imglist = codecs.open(imgdir, 'r')

    for line in imglist:  
        # Load the image
        tmp = line.strip().split(' ')
        try:
            img = load_image(tmp[0])
            label = int(tmp[1])
        except:
            continue

        # Create a feature
        feature = {'train/label': _int64_feature(label),
                   'train/image': _bytes_feature(tf.compat.as_bytes(img.tostring()))}

        # Create an example protocol buffer
        example = tf.train.Example(features=tf.train.Features(feature=feature))

        # Serialize to string and write on the file
        writer.write(example.SerializeToString())

    writer.close()
    sys.stdout.flush()

### 从生成的tfrecords文件中读取文件

In [ ]:
def read_and_decode(tf_filepath):
    
    # Create a list of filenames and pass it to a queue
    tfrecords_files = glob.glob(tf_filepath+'*.tfrecords')
    filename_queue = tf.train.string_input_producer(tfrecords_files)
    
    # Define a reader and read the next record
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    
    # Decode the record read by the reader
    features = tf.parse_single_example(serialized_example, 
                                       features={'train/image': tf.FixedLenFeature([], tf.string),
                                                 'train/label': tf.FixedLenFeature([], tf.int64)})
    # Convert the image data from string back to the numbers
    image = tf.decode_raw(features['train/image'], tf.float32)

    # Cast label data into int32
    label = tf.cast(features['train/label'], tf.int32)
    # Reshape image data into the original shape
    image = tf.reshape(image, [224, 224, 3])

    # Any preprocessing here ...

    # Creates batches by randomly shuffling tensor
    images, labels = tf.train.batch([image, label], batch_size=500, capacity=2000, num_threads=64)

    return images, tf.one_hot(tf.subtract(labels, 1), 5)